In [1]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
print("Setup Complete")
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file
from sklearn.metrics import mean_squared_error


def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def classificateWithNumbersThreeTarget(value):
    target=0
    if(value>=0.0015):
        target=1
    elif(value<=-0.0015):
        target=-1
    else:
        target=0
    return target


def kirajzoltatas(yhat,test_y):
    yhat = pd.DataFrame(yhat, columns=['Predicted'])
    test_y2 = pd.DataFrame(test_y, columns=['Real'])
    merge=test_y2.join(yhat)

    import seaborn as sns
    import matplotlib.pyplot as plt

    plt.figure(figsize=(30,20))
    sns.lineplot(data=merge[:100], linewidth=3)
    plt.show()
    

def correlation_heatmap(df):
    correlations = df.corr()

    fig, ax = plt.subplots(figsize=(40,40))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.1, annot=True, cbar_kws={"shrink": .7})
    sns.set(font_scale=2)
    plt.show();

def aboveValue(openList,inspectList,df,name):
    results=[]
    for i in range(len(inspectList)):
        if (openList[i]>inspectList[i]):
            results.append(1)
        else:
            results.append(0)
    df[name]=results
    
data1 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_06', sep=",", header=None)
data1.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data2 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_07', sep=",", header=None)
data2.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data3 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_08', sep=",", header=None)
data3.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data4 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_09', sep=",", header=None)
data4.columns = ["time", "open", "high", "low","close","volume"]

data= pd.concat([data1,data2,data3,data4], axis=0)
#data

#data=data4
    

Setup Complete


In [ ]:
%%time
#btc_data = pd.read_csv(r"/home/balazs/Desktop/szakdoga_info/btc_data.csv")
#data = btc_data.dropna()
data['MA5']=data['open'].rolling(window =5).mean()
data['EMA5']=data['open'].ewm(span=5,min_periods=0,adjust=False,ignore_na=False).mean()
data['CMA']=data.open.expanding().mean()
data['open_percentage_change']=data.open.pct_change()
data['close_percentage_change']=data.close.pct_change()
data['high_percentage_change']=data.high.pct_change()
data['low_percentage_change']=data.low.pct_change()
data['volume_percentage_change']=data.volume.pct_change()
lista=[]
for i in range(len(data.open_percentage_change.values)-5):
        value=data.open_percentage_change.values[i+1]+data.open_percentage_change.values[i+2]+data.open_percentage_change.values[i+3]+data.open_percentage_change.values[i+4]+data.open_percentage_change.values[i+5]
        lista.append(value)
for i in range(5):
    lista.append(0)
data['profit']=lista
buy_sell_list=[classificateWithNumbersThreeTarget(elem) for elem in data.profit]
data['decision']=buy_sell_list
data=data.dropna()
data.drop(data.tail(5).index,inplace=True) # drop last n rows
aboveValue(data.open.values,data.MA5.values,data,'aboveMA5')
aboveValue(data.open.values,data.EMA5.values,data,'aboveEMA5')
aboveValue(data.open.values,data.CMA.values,data,'aboveCMA')

In [ ]:
data

In [ ]:
X=data[['close_percentage_change','high_percentage_change','low_percentage_change','decision']]
y=data[['decision']]

from imblearn.combine import SMOTETomek
smt = SMOTETomek(random_state=42)
X, y = smt.fit_resample(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [ ]:
X

In [ ]:
y

# tanítás

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [ ]:
data

In [ ]:
lista=data['profit'].values
decision_list=data['decision'].values

In [ ]:
n = len(lista)
data_train = lista[0 : int(n*0.8)]
data_test = lista[int(n*0.8):]

test_decision_list=decision_list[int(n*0.8):]

In [ ]:
np.unique(test_decision_list,return_counts=True)

In [ ]:
test_decision_list[149]

In [ ]:
data_test[149]


In [ ]:
# choose a number of time steps
n_steps = 10
# split into samples

train_X, train_y= split_sequence(data_train, n_steps)
# summarize the data


In [ ]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], n_features))

In [ ]:
tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
# This callback will stop the training when there is no improvement in  
# the validation loss for three consecutive epochs.  
model = Sequential()
model.add(LSTM(25, activation='relu', input_shape=(10, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
history=model.fit(train_X, train_y, epochs=20, verbose=1, callbacks=[callback])
len(history.history['loss'])

In [ ]:
test_decision_list

In [ ]:
# choose a number of time steps
n_steps = 10
# split into samples
test_X, test_y = split_sequence(data_test, n_steps)
# summarize the data

In [ ]:
# choose a number of time steps
n_steps = 10
# split into samples
test_dec_X, test_dec_y = split_sequence(test_decision_list, n_steps)
# summarize the data

In [ ]:
test_dec_y

In [ ]:
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], n_features))

In [ ]:
test_dec_X = test_dec_X.reshape((test_dec_X.shape[0], test_dec_X.shape[1], n_features))

In [ ]:
test_dec_X

In [ ]:
yhat = model.predict(test_X, verbose=0)

In [ ]:
performance =model.evaluate(test_X,test_y)
print (performance)

In [ ]:
mean_squared_error(test_y, yhat)

In [ ]:
def metrikak(test_y,yhat):
    print(f"""MSE:{mean_squared_error(test_y, yhat)}""")
    #print(f"""MSE:{mean_squared_error(test_y, yhat)}""")
    #print(f"""MSE:{mean_squared_error(test_y, yhat)}""")

In [ ]:
kirajzoltatas(yhat,test_y)

In [ ]:
test_y

In [ ]:
test_dec_y

In [ ]:
joslatok=[elem[0] for elem in yhat]

In [ ]:
joslatok_decisionben=[classificateWithNumbersThreeTarget(elem) for elem in joslatok]

In [ ]:
joslatok_decisionben

In [ ]:
from sklearn.metrics import classification_report
classes = ['-1','0','1']
print(classification_report(test_dec_y, joslatok_decisionben, target_names = classes, zero_division = 0))